In [1]:
import os
# choose which GPU 
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "1"

import pandas as pd
from keras.callbacks import ModelCheckpoint, CSVLogger
from keras.optimizers import Adam, RMSprop
from keras.callbacks import TensorBoard
import datetime
import time

from generator import BRATS_DataGenerator
from losses import *
from vnet import vnet

Using TensorFlow backend.


In [2]:
model = vnet(192,192,192, summary=False) #set summary to True to see the model architecture 

W0706 08:46:38.797799 140477356586816 deprecation_wrapper.py:119] From /home/Shahryar.Khorasani/anaconda3/envs/clone_keras/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0706 08:46:38.807947 140477356586816 deprecation_wrapper.py:119] From /home/Shahryar.Khorasani/anaconda3/envs/clone_keras/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0706 08:46:38.809978 140477356586816 deprecation_wrapper.py:119] From /home/Shahryar.Khorasani/anaconda3/envs/clone_keras/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0706 08:46:38.820858 140477356586816 deprecation_wrapper.py:119] From /home/Shahryar.Khorasani/anaconda3/envs/clone_keras/lib/python3.6/site-packages/keras/back

____________________________________________________________________________________________________________________________________________
Layer (type)                                  Output Shape                   Param #         Connected to                                   
input_1 (InputLayer)                          (None, 192, 192, 192, 4)       0                                                              
____________________________________________________________________________________________________________________________________________
conv3d_1 (Conv3D)                             (None, 192, 192, 192, 18)      1962            input_1[0][0]                                  
____________________________________________________________________________________________________________________________________________
leaky_re_lu_1 (LeakyReLU)                     (None, 192, 192, 192, 18)      0               conv3d_1[0][0]                                 
_____________

In [3]:
brats = pd.read_csv('/mnt/dsets/brats/survival_data.csv')
# the first 100 subjects are used as training 
gen_train = BRATS_DataGenerator(list(brats.BraTS18ID[:100]), augment_flip=True, seed = 2020) 
gen_valid = BRATS_DataGenerator(list(brats.BraTS18ID[100:]), augment_flip=False)

# We should try K-fold cross validation since there are so few examples (only 163 subjects)

In [ ]:
stemdir = '/your directory of choice/{}'.format(datetime.datetime.fromtimestamp(time.time()).strftime('%Y_%m_%d_%H%M%S'))
csv_dir = stemdir + '/vnet_brats.csv'
checkpoint_dir = stemdir + '/vnet_brats_{}.h5'.format(datetime.datetime.fromtimestamp(time.time()).strftime('%Y_%m_%d_%H%M%S'))

In [ ]:
if not os.path.exists(stemdir):
    os.makedirs(stemdir)
# logger files    
model_checkpoint = ModelCheckpoint(checkpoint_dir, monitor='val_loss', verbose=1, save_best_only=True)
csv_logger = CSVLogger(csv_dir) 
tb_logger = TensorBoard(log_dir=stemdir, write_grads=True, batch_size=1, update_freq='batch')

model.compile(optimizer=RMSprop(lr=0.000015, decay=0.0002), loss=generalised_dice_loss_3D, metrics=['accuracy', brats_wt_metric, brats_tc_metric, brats_et_metric])
# fit_generator works with generators that  written in keras.utils.Sequence format. It allows multiprocessing, so the model trains faster.
model.fit_generator(gen_train, validation_data=gen_valid, epochs=500, verbose=1, callbacks=[model_checkpoint, csv_logger, tb_logger], max_queue_size=10,
                        workers=8, use_multiprocessing=True)